Lets convert the datacard to a binary workspace

In [2]:
%%bash
text2workspace.py -m 125 comb_2017_htt_slim.txt -o comb_2017_htt_slim.root --channel-masks

Channel htt_em_1_13TeV will use autoMCStats with settings: event-threshold=0, include-signal=0, hist-mode=1
Analysing bin errors for: prop_binhtt_em_1_13TeV
Poisson cut-off: 0
Processes excluded for sums: qqH_htt qqH_hww ggH_htt ZH_lep_htt ggH_hww WH_lep_htt WH_had_htt ZH_had_htt
Bin        Contents        Error           Notes                         
0          12923.279248    131.535886      total sum                     
0          12888.213882    131.531897      excluding marked processes    
0          9601.000000     97.984693       Unweighted events, alpha=1.346035
  => Total parameter prop_binhtt_em_1_13TeV_bin0[0.00,-7.00,7.00] to be gaussian constrained
------------------------------------------------------------
1          8122.601386     104.907730      total sum                     
1          8099.800424     104.903020      excluding marked processes    
1          5962.000000     77.213988       Unweighted events, alpha=1.362395
  => Total parameter prop_binhtt_em_1_13T

We will now run the multi dimensional fit. Strategy 0 is with no covariance matrix, and strategy 1 is with covariance matrix. We also configure a few fallbak algorithms in case the fit has trouble converging. We are using the analytic minimization of bin-by-bin MC statistical uncertainties.

In [4]:
%%bash
combine -M MultiDimFit -m 125.09 -d comb_2017_htt_slim.root -n BestFit_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --saveToys --saveWorkspace --saveFitResult

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters

 --- MultiDimFit ---
best fit parameter values: 
   r :    +1.212
Done in 0.34 min (cpu), 0.35 min (real)


Lets now load best fit snapshot from step 1 and compute uncertainties using algo singles. We use robustFit option to get the best estimate of the uncertatinties.

In [7]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFit_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -n mu_htt_slim_AllUnc --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --algo=singles --robustFit=1

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
 minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters

 --- MultiDimFit ---
best fit parameter values and profile-likelihood uncertainties: 
   r :    +1.212   -0.262/+0.281 (68%)
Done in 1.82 min (cpu), 1.82 min (r

Lets again load best fit snapshot from step 1 and compute uncertainties from step 1 using algo singles but this time, lets freeze all constrained nuisance parameters. This will give us the "statistical only" uncertainty.

In [9]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFit_htt_slim.MultiDimFit.mH125.09.123456.root  -w w --snapshotName "MultiDimFit" -n mu_htt_slim_StatOnly --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --algo=singles --robustFit=1 --freezeParameters all

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical u

Lets load best fit snapshot from step 1 and now compute uncertainties using algo grid with auto range. This is the "likelihood scan".

In [10]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFit_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -n mu_htt_slim_ScanAllUnc --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --algo=grid  --autoRange 3 --points 30 

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Adjusting range of POIs to +/- 3 standard deviations
r: 1.21197 +/- 0.265008 [ 0.3 , 1.7 ] ==> [ 0.41694 , 1.7 ]
 POI: r= 1.21197 -> [0.41694,1.7]
Point 0/30 r = 0.438325
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Po

Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max


Lets load best fit snapshot from step 1 and compute uncertainties using algo grid with auto range but this time freeze all constrained nuisance parameters. This will give the "stat only" uncertainty.

In [8]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFit_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -n mu_htt_slim_ScanStatOnly --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --skipInitialFit --algo=grid  --autoRange 3 --points 30 --freezeParameters all

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
MultiDimFit -- Skipping initial global fit
Adjusting range of POIs to +/- 3 standard deviations
r: 1.21197 +/- 0.264316 [ 0.3 , 1.7 ] ==> [ 0.419027 , 1.7 ]
 POI: r= 1.21197 -> [0.419027,1.7]
Point 0/30 r = 0.440377
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Point 1/30 r = 0.483076
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling an

Lets use a standard script provided in CombineHarvester to plot the 1D scans breaking down statistical and systematic components

In [4]:
%%bash
plot1DScan.py higgsCombinemu_htt_slim_ScanAllUnc.MultiDimFit.mH125.09.root --main-label "Observed" --others "higgsCombinemu_htt_slim_ScanStatOnly.MultiDimFit.mH125.09.root:Stat. Only:2" --POI r --breakdown syst,stat --output scan_htt_r

--------------------------------------
scan_htt_r
--------------------------------------
[{'lo': 0.9486553250921668, 'hi': 1.4950284566048146, 'valid_hi': True, 'valid_lo': True}]
[{'lo': 0.7036001446200941, 'hi': 1.678615689277649, 'valid_hi': False, 'valid_lo': True}]
[{'lo': 1.0640280878188617, 'hi': 1.3617013302645171, 'valid_hi': True, 'valid_lo': True}]
[{'lo': 0.9179248781014657, 'hi': 1.513120355463922, 'valid_hi': True, 'valid_lo': True}]


Info in <TCanvas::Print>: pdf file ./scan_htt_r.pdf has been created
Info in <TCanvas::Print>: png file ./scan_htt_r.png has been created


In [5]:
from IPython.display import IFrame
IFrame("./scan_htt_r.pdf", width=800, height=800)

Now we will start doing prefit asimov studies. We will start with generating a prefit asimov dataset with r=1 and then fitting it

In [15]:
%%bash
combine -M MultiDimFit -m 125.09 -d comb_2017_htt_slim.root -n BestFitAsimov_htt_slim --setParameters r=1 -t -1 --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --saveToys --saveWorkspace --saveFitResult

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
Set Default Value of Parameter r To : 1
>>> 0 out of 24 channels masked

Computing results starting from expected outcome (a-priori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters

 --- MultiDimFit ---
best fit parameter values: 
   r :    +1.000
Done in 0.33 min (cpu), 0.33 min (real)


Lets load prefit asimov best fit snapshot from the previous step and compute uncertainties on "r" from prefit asimov dataset with r=1 using algo singles.

In [16]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_AsimovAllUnc --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --algo=singles --robustFit=1

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters

Lets load again the prefit asimov best fit snapshot from the previous step (7) and compute uncertainties on r from prefit asimov dataset with r=1 using algo singles but this time lets freeze constrained nuisance parameters

In [17]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_AsimovStatOnly --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --algo=singles --robustFit=1 --freezeParameters all

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical u

Lets load prefit asimov best fit snapshot from the previous step (7) and compute uncertainties on r from prefit asimov dataset with r=1 using algo singles

In [18]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_ScanAsimovAllUnc --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --skipInitialFit --algo=grid  --autoRange 3 --points 30

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
MultiDimFit -- Skipping initial global fit
Adjusting range of POIs to +/- 3 standard deviations
r: 1.00001 +/- 0.187023 [ 0.3 , 1.7 ] ==> [ 0.438937 , 1.56108 ]
 POI: r= 1.00001 -> [0.438937,1.56108]
Point 0/30 r = 0.45764
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Point 1/30 r = 0.495044
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disab

Lets again load prefit asimov best fit snapshot from the previous step (7) and compute uncertainties on r from prefit asimov dataset with r=1 using algo singles but this time freeze constrained NPs

In [19]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_ScanAsimovStatOnly --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --skipInitialFit --algo=grid --autoRange 3 --points 30 --freezeParameters all

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
MultiDimFit -- Skipping initial global fit
Adjusting range of POIs to +/- 3 standard deviations
r: 1.00001 +/- 0.187023 [ 0.3 , 1.7 ] ==> [ 0.438937 , 1.56108 ]
 POI: r= 1.00001 -> [0.438937,1.56108]
Point 0/30 r = 0.45764
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Point 1/30 r = 0.495044
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disab

Lets now plot the 1D likelihood scans breaking down stat. and syst. components

In [6]:
%%bash
plot1DScan.py higgsCombinemu_htt_slim_ScanAsimovAllUnc.MultiDimFit.mH125.09.root --main-label "Expected" --others "higgsCombinemu_htt_slim_ScanAsimovStatOnly.MultiDimFit.mH125.09.root:Stat. Only:2" --POI r --breakdown syst,stat --output asimovscan_htt_r

--------------------------------------
asimovscan_htt_r
--------------------------------------
[{'lo': 0.7646294036383148, 'hi': 1.252368209450535, 'valid_hi': True, 'valid_lo': True}]
[{'lo': 0.5452439534778506, 'hi': 1.5248875728894538, 'valid_hi': True, 'valid_lo': True}]
[{'lo': 0.860518512287296, 'hi': 1.1411365706528653, 'valid_hi': True, 'valid_lo': True}]
[{'lo': 0.7230131853923466, 'hi': 1.284066982065768, 'valid_hi': True, 'valid_lo': True}]


Info in <TCanvas::Print>: pdf file ./asimovscan_htt_r.pdf has been created
Info in <TCanvas::Print>: png file ./asimovscan_htt_r.png has been created


In [7]:
from IPython.display import IFrame
IFrame("./asimovscan_htt_r.pdf", width=800, height=800)

Now lets switch to postfit asimov studies and generate postfit asimov dataset with r=1 and fit it. The main difference here is the option "--toysFrequentist" which makes it post-fit.


In [22]:
%%bash
combine -M MultiDimFit -m 125.09 -d comb_2017_htt_slim.root -n BestFitPostfitAsimov_htt_slim --setParameters r=1 -t -1 --toysFrequentist --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --saveToys --saveWorkspace --saveFitResult

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
Set Default Value of Parameter r To : 1
>>> 0 out of 24 channels masked

>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Computing results starting from expected outcome (a-priori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters

 --- MultiDimFit ---
best fit parameter values: 
   r :    +1.000
Done in 0.32 min (cpu), 0.32 min (real)


Lets load postfit asimov best fit snapshot from the previous step and compute uncertainties on r from postfit asimov dataset with r=1 using algo singles

In [23]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitPostfitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_PostfitAsimovAllUnc --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --algo=singles --robustFit=1

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters

Now lets load postfit asimov best fit snapshot from the previous step and compute uncertainties on r from postfit asimov dataset with r=1 using algo singles but freeze constrained NPs

In [24]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitPostfitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_PostfitAsimovStatOnly --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --algo=singles --robustFit=1 --freezeParameters all

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical u

Lets load postfit asimov best fit snapshot from the previous step and compute uncertainties on r from postfit asimov dataset with r=1 using algo grid

In [25]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitPostfitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_ScanPostfitAsimovAllUnc --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --skipInitialFit --algo=grid  --autoRange 3 --points 30

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
MultiDimFit -- Skipping initial global fit
Adjusting range of POIs to +/- 3 standard deviations
r: 1 +/- 0.1107 [ 0.3 , 1.7 ] ==> [ 0.667903 , 1.3321 ]
 POI: r= 1 -> [0.667903,1.3321]
Point 0/30 r = 0.678973
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Point 1/30 r = 0.701113
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic m

Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max


Lets load postfit asimov best fit snapshot from the previous step and compute uncertainties on r from postfit asimov dataset with r=1 using algo grid but freeze constrained NPs to get the stat only uncertainty

In [26]:
%%bash
combine -M MultiDimFit -m 125.09 -d higgsCombineBestFitPostfitAsimov_htt_slim.MultiDimFit.mH125.09.123456.root -w w --snapshotName "MultiDimFit" -D toys/toy_asimov -n mu_htt_slim_ScanPostfitAsimovStatOnly --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --skipInitialFit --algo=grid --autoRange 3 --points 30 --freezeParameters all

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
MultiDimFit -- Skipping initial global fit
Adjusting range of POIs to +/- 3 standard deviations
r: 1 +/- 0.1107 [ 0.3 , 1.7 ] ==> [ 0.667903 , 1.3321 ]
 POI: r= 1 -> [0.667903,1.3321]
Point 0/30 r = 0.678973
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
Point 1/30 r = 0.701113
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic m

Lets now plot the 1D likelihood scans from postfit asimov breaking down stat. and syst. components

In [8]:
%%bash
plot1DScan.py higgsCombinemu_htt_slim_ScanPostfitAsimovAllUnc.MultiDimFit.mH125.09.root --main-label "Post-Fit Expected" --others "higgsCombinemu_htt_slim_ScanPostfitAsimovStatOnly.MultiDimFit.mH125.09.root:Stat. Only:2" --POI r --breakdown syst,stat --output postfitasimovscan_htt_r

--------------------------------------
postfitasimovscan_htt_r
--------------------------------------
[{'lo': 0.7590498299839584, 'hi': 1.2581819220442203, 'valid_hi': True, 'valid_lo': True}]
[{'lo': 0.6789727807044983, 'hi': 1.3210341930389404, 'valid_hi': False, 'valid_lo': False}]
[{'lo': 0.8573208874751277, 'hi': 1.144377594625135, 'valid_hi': True, 'valid_lo': True}]
[{'lo': 0.7165314735077633, 'hi': 1.2904894518728414, 'valid_hi': True, 'valid_lo': True}]


Info in <TCanvas::Print>: pdf file ./postfitasimovscan_htt_r.pdf has been created
Info in <TCanvas::Print>: png file ./postfitasimovscan_htt_r.png has been created


In [10]:
from IPython.display import IFrame
IFrame("./postfitasimovscan_htt_r.pdf", width=800, height=800)

Now we will look into doing impact studies, starting with the expected impacts. The impact of a nuisance parameter (NP) θ on a parameter of interest (POI) μ is defined as the shift Δμ that is induced as θ is fixed and brought to its +1σ or −1σ post-fit values, with all other parameters profiled as normal. This is effectively a measure of the correlation between the NP and the POI, and is useful for determining which NPs have the largest effect on the POI uncertainty. We start this with the prefit asimov.

In [30]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n ImpactsAsimov_htt_slim --setParameters r=1 -t -1 --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 0 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH  --doInitialFit --robustFit 1

Have POIs: ['r']
 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
Set Default Value of Parameter r To : 1
>>> 0 out of 24 channels masked

Computing results starting from expected outcome (a-priori)
minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic m

Lets run the impacts but only for a subset of the nuisance parameters (using the option --named"

In [31]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n ImpactsAsimov_htt_slim --setParameters r=1 -t -1 --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --doFits --robustFit 1 --parallel 8 --named BR_htt_PU_alphas,BR_htt_PU_mq,BR_htt_THU,BR_hww_PU_alphas,BR_hww_PU_mq,BR_hww_THU,CMS_PS,CMS_eFakeTau_1prong1pizero_13TeV,CMS_eFakeTau_1prong_13TeV,CMS_eff_b_13TeV,CMS_eff_t_13TeV,CMS_eff_t_et_13TeV,CMS_eff_t_mt_13TeV,CMS_eff_t_tt_13TeV,CMS_eff_trigger_em_13TeV,CMS_eff_trigger_et_13TeV,CMS_eff_trigger_mt_13TeV,CMS_eff_trigger_tt_13TeV,CMS_htt_QCD_0jet_em_13TeV,CMS_htt_QCD_0jet_tt_13TeV,CMS_htt_QCD_VBF_em_13TeV,CMS_htt_QCD_VBF_tt_13TeV,CMS_htt_QCD_boosted_em_13TeV,CMS_htt_QCD_boosted_tt_13TeV,CMS_htt_eff_e,CMS_htt_eff_m,CMS_htt_jetFakeLep_13TeV,CMS_htt_scale_met_13TeV,CMS_htt_vvXsec_13TeV,CMS_htt_wjXsec_13TeV,CMS_htt_zmm_norm_extrap_0jet_em_13TeV,CMS_htt_zmm_norm_extrap_0jet_lt_13TeV,CMS_htt_zmm_norm_extrap_0jet_tt_13TeV,CMS_htt_zmm_norm_extrap_VBF_em_13TeV,CMS_htt_zmm_norm_extrap_VBF_lt_13TeV,CMS_htt_zmm_norm_extrap_VBF_tt_13TeV,CMS_htt_zmm_norm_extrap_boosted_em_13TeV,CMS_htt_zmm_norm_extrap_boosted_lt_13TeV,CMS_htt_zmm_norm_extrap_boosted_tt_13TeV,CMS_mFakeTau_1prong1pizero_13TeV,CMS_mFakeTau_1prong_13TeV,CMS_scale_e_em_13TeV,CMS_tauDMReco_1prong1pizero_13TeV,CMS_tauDMReco_1prong_13TeV,CMS_tauDMReco_3prong_13TeV,QCD_Extrap_Iso_nonIso_et_13TeV,QCD_Extrap_Iso_nonIso_mt_13TeV,QCDscale_qqH,THU_ggH_Mig01,THU_ggH_Mig12,THU_ggH_PT120,THU_ggH_PT60,THU_ggH_Res,THU_ggH_VBF2j,THU_ggH_VBF3j,THU_ggH_qmtop,WHighMTtoLowMT_0jet_13TeV,WHighMTtoLowMT_boosted_13TeV,WHighMTtoLowMT_vbf_13TeV,WSFUncert_et_0jet_13TeV,WSFUncert_et_boosted_13TeV,WSFUncert_et_vbf_13TeV,WSFUncert_mt_0jet_13TeV,WSFUncert_mt_boosted_13TeV,WSFUncert_mt_vbf_13TeV,lumi_13TeV_2016,pdf_Higgs_gg,pdf_Higgs_qqbar

Have POIs: ['r']
Have parameters: 68
Setting paramter r to 1
 <<< Combine >>> 
 <<< Combine >>> 
 <<< Combine >>> 
 <<< Combine >>> 
 <<< Combine >>> 
 <<< Combine >>> 
>>> including systematics
>>> including systematics
>>> including systematics
>>> including systematics
>>> including systematics
>>> including systematics
   Options for Robust Minimizer :: 
   Options for Robust Minimizer :: 
   Options for Robust Minimizer :: 
   Options for Robust Minimizer :: 
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Tolerance  0.1
        Tolerance  0.1
        Tolerance  0.1
        Strategy   0
        Tolerance  0.1
   Options for Robust Minimizer :: 
        Type,Algo  Minuit2,Migrad
 <<< Combine >>> 
 <<< Combine >>> 
        Strategy   0
        Strategy   0
        Strategy   0
        Type,Algo  Minuit2,Migrad
        Strategy   0
        Type,Algo  Minuit2,Migrad
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
        Type,Algo  Minuit2,Migrad
>>

 Look at that last command above to get an idea of what combine is doing:
 
 combine -M MultiDimFit -n _paramFit_ImpactsAsimov_htt_slim_lumi_13TeV_2016 --algo impact --redefineSignalPOIs r -P lumi_13TeV_2016 --floatOtherPOIs 1 --saveInactivePOI 1 -t -1 --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --robustFit 1 -m 125.09 -d comb_2017_htt_slim.root --setParameters r=1
 
 We are treating lumi_13TeV_2016 as the POI with the "-P" option (i.e. findings its +/- sigma). We are using the option "--saveInactivePOI" to keep track of where the real poi, r, is at these points. 

Now lets merge the impacts results to an output file (only for a subset)

In [32]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n ImpactsAsimov_htt_slim --setParameters r=1 -t -1 --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH  -o impacts_asimov.json --named BR_htt_PU_alphas,BR_htt_PU_mq,BR_htt_THU,BR_hww_PU_alphas,BR_hww_PU_mq,BR_hww_THU,CMS_PS,CMS_eFakeTau_1prong1pizero_13TeV,CMS_eFakeTau_1prong_13TeV,CMS_eff_b_13TeV,CMS_eff_t_13TeV,CMS_eff_t_et_13TeV,CMS_eff_t_mt_13TeV,CMS_eff_t_tt_13TeV,CMS_eff_trigger_em_13TeV,CMS_eff_trigger_et_13TeV,CMS_eff_trigger_mt_13TeV,CMS_eff_trigger_tt_13TeV,CMS_htt_QCD_0jet_em_13TeV,CMS_htt_QCD_0jet_tt_13TeV,CMS_htt_QCD_VBF_em_13TeV,CMS_htt_QCD_VBF_tt_13TeV,CMS_htt_QCD_boosted_em_13TeV,CMS_htt_QCD_boosted_tt_13TeV,CMS_htt_eff_e,CMS_htt_eff_m,CMS_htt_jetFakeLep_13TeV,CMS_htt_scale_met_13TeV,CMS_htt_vvXsec_13TeV,CMS_htt_wjXsec_13TeV,CMS_htt_zmm_norm_extrap_0jet_em_13TeV,CMS_htt_zmm_norm_extrap_0jet_lt_13TeV,CMS_htt_zmm_norm_extrap_0jet_tt_13TeV,CMS_htt_zmm_norm_extrap_VBF_em_13TeV,CMS_htt_zmm_norm_extrap_VBF_lt_13TeV,CMS_htt_zmm_norm_extrap_VBF_tt_13TeV,CMS_htt_zmm_norm_extrap_boosted_em_13TeV,CMS_htt_zmm_norm_extrap_boosted_lt_13TeV,CMS_htt_zmm_norm_extrap_boosted_tt_13TeV,CMS_mFakeTau_1prong1pizero_13TeV,CMS_mFakeTau_1prong_13TeV,CMS_scale_e_em_13TeV,CMS_tauDMReco_1prong1pizero_13TeV,CMS_tauDMReco_1prong_13TeV,CMS_tauDMReco_3prong_13TeV,QCD_Extrap_Iso_nonIso_et_13TeV,QCD_Extrap_Iso_nonIso_mt_13TeV,QCDscale_qqH,THU_ggH_Mig01,THU_ggH_Mig12,THU_ggH_PT120,THU_ggH_PT60,THU_ggH_Res,THU_ggH_VBF2j,THU_ggH_VBF3j,THU_ggH_qmtop,WHighMTtoLowMT_0jet_13TeV,WHighMTtoLowMT_boosted_13TeV,WHighMTtoLowMT_vbf_13TeV,WSFUncert_et_0jet_13TeV,WSFUncert_et_boosted_13TeV,WSFUncert_et_vbf_13TeV,WSFUncert_mt_0jet_13TeV,WSFUncert_mt_boosted_13TeV,WSFUncert_mt_vbf_13TeV,lumi_13TeV_2016,pdf_Higgs_gg,pdf_Higgs_qqbar

Have POIs: ['r']
Have parameters: 68
Setting paramter r to 1


Lets now make the plot of the expected impacts for a subset

In [34]:
%%bash
plotImpacts.py -i impacts_asimov.json -o impacts_asimov

>> Doing page 0, have 30 parameters
>> Doing page 1, have 30 parameters
>> Doing page 2, have 8 parameters


Info in <TCanvas::Print>: pdf file ./impacts_asimov.pdf has been created
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: impacts_asimov
Info in <TCanvas::Print>: Current canvas added to pdf file ./impacts_asimov.pdf
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: impacts_asimov
Info in <TCanvas::Print>: Current canvas added to pdf file ./impacts_asimov.pdf


In [11]:
from IPython.display import IFrame
IFrame("./impacts_asimov.pdf", width=800, height=800)

You can see that we are expecting in this fit to constrain some nuisance parameters. In your analysis, you will need to justify whether or not these constraints are expected. In some cases the nuisance parameters are estimated conservatively (e.g. the fake rate extrapolation). In others the POG provided uncertainties were actually done less precisely that what can be learned from this dataset! That is the case for the tau energy scale uncertainties. There are certain uncertainties that should never get constrained: e.g. luminosity. If so you need to go talk to the lumi POG and tell them why you can measure the luminosity better than they can! This is very important to go through this list and make sure you understand every thing about it. Sometimes, the nuisance parameters very far down the list can be ignored if they are constrained because they do not affect the result. However you need to be careful if you are combining with another measurement and that nuisance parameter is important in that channel!

Now let us re-run the impacts with the observed data for a subset of nuisances, starting with the initial fit

In [12]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n Impacts_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH  --doInitialFit --robustFit 1

Have POIs: ['r']
 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
   Options for Robust Minimizer :: 
        Tolerance  0.1
        Strategy   0
        Type,Algo  Minuit2,Migrad
>>> method used is MultiDimFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncer

Now lets do the impact fits on the observed dataset for a subset of the NPs. This will take upwards of 30 minutes. If you want to skip it you can proceeed to the next step instead of running it. 

In [13]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n Impacts_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --doFits --robustFit 1 --parallel 8 --named BR_htt_PU_alphas,BR_htt_PU_mq,BR_htt_THU,BR_hww_PU_alphas,BR_hww_PU_mq,BR_hww_THU,CMS_PS,CMS_eFakeTau_1prong1pizero_13TeV,CMS_eFakeTau_1prong_13TeV,CMS_eff_b_13TeV,CMS_eff_t_13TeV,CMS_eff_t_et_13TeV,CMS_eff_t_mt_13TeV,CMS_eff_t_tt_13TeV,CMS_eff_trigger_em_13TeV,CMS_eff_trigger_et_13TeV,CMS_eff_trigger_mt_13TeV,CMS_eff_trigger_tt_13TeV,CMS_htt_QCD_0jet_em_13TeV,CMS_htt_QCD_0jet_tt_13TeV,CMS_htt_QCD_VBF_em_13TeV,CMS_htt_QCD_VBF_tt_13TeV,CMS_htt_QCD_boosted_em_13TeV,CMS_htt_QCD_boosted_tt_13TeV,CMS_htt_eff_e,CMS_htt_eff_m,CMS_htt_jetFakeLep_13TeV,CMS_htt_scale_met_13TeV,CMS_htt_vvXsec_13TeV,CMS_htt_wjXsec_13TeV,CMS_htt_zmm_norm_extrap_0jet_em_13TeV,CMS_htt_zmm_norm_extrap_0jet_lt_13TeV,CMS_htt_zmm_norm_extrap_0jet_tt_13TeV,CMS_htt_zmm_norm_extrap_VBF_em_13TeV,CMS_htt_zmm_norm_extrap_VBF_lt_13TeV,CMS_htt_zmm_norm_extrap_VBF_tt_13TeV,CMS_htt_zmm_norm_extrap_boosted_em_13TeV,CMS_htt_zmm_norm_extrap_boosted_lt_13TeV,CMS_htt_zmm_norm_extrap_boosted_tt_13TeV,CMS_mFakeTau_1prong1pizero_13TeV,CMS_mFakeTau_1prong_13TeV,CMS_scale_e_em_13TeV,CMS_tauDMReco_1prong1pizero_13TeV,CMS_tauDMReco_1prong_13TeV,CMS_tauDMReco_3prong_13TeV,QCD_Extrap_Iso_nonIso_et_13TeV,QCD_Extrap_Iso_nonIso_mt_13TeV,QCDscale_qqH,THU_ggH_Mig01,THU_ggH_Mig12,THU_ggH_PT120,THU_ggH_PT60,THU_ggH_Res,THU_ggH_VBF2j,THU_ggH_VBF3j,THU_ggH_qmtop,WHighMTtoLowMT_0jet_13TeV,WHighMTtoLowMT_boosted_13TeV,WHighMTtoLowMT_vbf_13TeV,WSFUncert_et_0jet_13TeV,WSFUncert_et_boosted_13TeV,WSFUncert_et_vbf_13TeV,WSFUncert_mt_0jet_13TeV,WSFUncert_mt_boosted_13TeV,WSFUncert_mt_vbf_13TeV,lumi_13TeV_2016,pdf_Higgs_gg,pdf_Higgs_qqbar

Process is terminated.


Now lets merge the observed impacts on a subset of NPs. In some cases the Impacts did not converge (this can happen, especially when running multicore). Normally you should rerun them, but in this case I have just dropped them.

In [21]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n Impacts_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH  -o impacts.json --named BR_htt_PU_alphas,BR_htt_PU_mq,BR_htt_THU,BR_hww_PU_alphas,BR_hww_PU_mq,BR_hww_THU,CMS_PS,CMS_eFakeTau_1prong1pizero_13TeV,CMS_eFakeTau_1prong_13TeV,CMS_eff_b_13TeV,CMS_eff_t_13TeV,CMS_eff_t_et_13TeV,CMS_eff_t_mt_13TeV,CMS_eff_t_tt_13TeV,CMS_eff_trigger_em_13TeV,CMS_eff_trigger_et_13TeV,CMS_eff_trigger_mt_13TeV,CMS_eff_trigger_tt_13TeV,CMS_htt_QCD_0jet_em_13TeV,CMS_htt_QCD_VBF_tt_13TeV,CMS_htt_QCD_boosted_em_13TeV,CMS_htt_QCD_boosted_tt_13TeV,CMS_htt_eff_e,CMS_htt_eff_m,CMS_htt_jetFakeLep_13TeV,CMS_htt_scale_met_13TeV,CMS_htt_vvXsec_13TeV,CMS_htt_wjXsec_13TeV,CMS_htt_zmm_norm_extrap_0jet_em_13TeV,CMS_htt_zmm_norm_extrap_0jet_lt_13TeV,CMS_htt_zmm_norm_extrap_0jet_tt_13TeV,CMS_htt_zmm_norm_extrap_VBF_em_13TeV,CMS_htt_zmm_norm_extrap_VBF_lt_13TeV,CMS_htt_zmm_norm_extrap_VBF_tt_13TeV,CMS_htt_zmm_norm_extrap_boosted_em_13TeV,CMS_htt_zmm_norm_extrap_boosted_lt_13TeV,CMS_htt_zmm_norm_extrap_boosted_tt_13TeV,CMS_mFakeTau_1prong1pizero_13TeV,CMS_mFakeTau_1prong_13TeV,CMS_scale_e_em_13TeV,CMS_tauDMReco_1prong1pizero_13TeV,CMS_tauDMReco_1prong_13TeV,CMS_tauDMReco_3prong_13TeV,QCD_Extrap_Iso_nonIso_et_13TeV,QCD_Extrap_Iso_nonIso_mt_13TeV,QCDscale_qqH,THU_ggH_Mig01,THU_ggH_Mig12,THU_ggH_PT120,THU_ggH_PT60,THU_ggH_Res,THU_ggH_VBF2j,THU_ggH_VBF3j,THU_ggH_qmtop,WHighMTtoLowMT_0jet_13TeV,WHighMTtoLowMT_boosted_13TeV,WHighMTtoLowMT_vbf_13TeV,WSFUncert_et_0jet_13TeV,WSFUncert_et_boosted_13TeV,WSFUncert_et_vbf_13TeV,WSFUncert_mt_0jet_13TeV,WSFUncert_mt_boosted_13TeV,WSFUncert_mt_vbf_13TeV,lumi_13TeV_2016,pdf_Higgs_gg,pdf_Higgs_qqbar

Have POIs: ['r']
Have parameters: 66
File higgsCombine_paramFit_Impacts_htt_slim_CMS_htt_QCD_boosted_em_13TeV.MultiDimFit.mH125.09.root did not contain a sufficient number of entries, skipping
File higgsCombine_paramFit_Impacts_htt_slim_CMS_htt_QCD_boosted_tt_13TeV.MultiDimFit.mH125.09.root did not contain a sufficient number of entries, skipping
File higgsCombine_paramFit_Impacts_htt_slim_CMS_htt_zmm_norm_extrap_boosted_tt_13TeV.MultiDimFit.mH125.09.root did not contain a sufficient number of entries, skipping
File higgsCombine_paramFit_Impacts_htt_slim_CMS_mFakeTau_1prong1pizero_13TeV.MultiDimFit.mH125.09.root did not contain a sufficient number of entries, skipping
File higgsCombine_paramFit_Impacts_htt_slim_CMS_mFakeTau_1prong_13TeV.MultiDimFit.mH125.09.root did not contain a sufficient number of entries, skipping
File higgsCombine_paramFit_Impacts_htt_slim_CMS_tauDMReco_3prong_13TeV.MultiDimFit.mH125.09.root did not contain a sufficient number of entries, skipping
Missing inputs: 

and plot the results of the impacts on observed for a subset

In [22]:
%%bash
plotImpacts.py -i impacts.json -o impacts

>> Doing page 0, have 30 parameters
>> Doing page 1, have 30 parameters


Info in <TCanvas::Print>: pdf file ./impacts.pdf has been created
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: impacts
Info in <TCanvas::Print>: Current canvas added to pdf file ./impacts.pdf


In [23]:
from IPython.display import IFrame
IFrame("./impacts.pdf", width=800, height=800)

You can see that some nuisance parameters are pulled away from their nominal expectation, and the constraints are a bit different in the observed data. You should understand these as well. For example if you have a nuisance parameter in two different channels, it shouldn't be pulled in opposite directions. Related to this topic is the goodness of fit which we will talk about later.

Now that we have seen the example of how to run the impacts on a subset of nuisances, you can try the full example on the grid system. This will send outputs to T3_US_FNALLPC. So it you will not be able to get the outputs in this Jupyter hub. This portion of the excercise is better done directly on the cms-lpc cluster. The produced output is copied here so that you can still look at the results.

We will start with the initial fit for full observed impacts. 

In [ ]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n FullImpacts_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH  --doInitialFit --robustFit 1 

You can submit the full osbserved impacts to crab. Make sure you have initialized your grid proxy and that you have sourced the crab environment

In [ ]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n FullImpacts_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --doFits --robustFit 1 --robustHesse 1  --job-mode crab3 --task-name full_impacts --custom-crab custom_crab.py

You can get the crab results of the impacts this way:

In [39]:
%%bash
#bash -c 'for f in /eos/uscms/store/user/${username}/Combine_Tutorial_2018/Combine/full_impacts/*/*/combine_output_*.tar; do tar xf $f; done'

tar: /eos/uscms/store/user//Combine_Tutorial_2018/Combine/full_impacts/*/*/combine_output_*.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Lets again merge the results of the impacts

In [40]:
%%bash
combineTool.py -M Impacts -m 125.09 -d comb_2017_htt_slim.root -n FullImpacts_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH  -o fullimpacts.json 

Have POIs: ['r']
Have parameters: 616
Missing inputs: prop_binhtt_tt_2_13TeV_bin30,prop_binhtt_tt_2_13TeV_bin9


Error in <TFile::TFile>: file higgsCombine_paramFit_FullImpacts_htt_slim_prop_binhtt_tt_2_13TeV_bin30.MultiDimFit.mH125.09.root does not exist
Error in <TFile::TFile>: file higgsCombine_paramFit_FullImpacts_htt_slim_prop_binhtt_tt_2_13TeV_bin9.MultiDimFit.mH125.09.root does not exist


And plot the full observed impactsc (over 600 nuisance parameters)

In [41]:
%%bash
plotImpacts.py -i fullimpacts.json -o fullimpacts

>> Doing page 0, have 30 parameters
>> Doing page 1, have 30 parameters
>> Doing page 2, have 30 parameters
>> Doing page 3, have 30 parameters
>> Doing page 4, have 30 parameters
>> Doing page 5, have 30 parameters
>> Doing page 6, have 30 parameters
>> Doing page 7, have 30 parameters
>> Doing page 8, have 30 parameters
>> Doing page 9, have 30 parameters
>> Doing page 10, have 30 parameters
>> Doing page 11, have 30 parameters
>> Doing page 12, have 30 parameters
>> Doing page 13, have 30 parameters
>> Doing page 14, have 30 parameters
>> Doing page 15, have 30 parameters
>> Doing page 16, have 30 parameters
>> Doing page 17, have 30 parameters
>> Doing page 18, have 30 parameters
>> Doing page 19, have 30 parameters
>> Doing page 20, have 14 parameters


Info in <TCanvas::Print>: pdf file ./fullimpacts.pdf has been created
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: fullimpacts
Info in <TCanvas::Print>: Current canvas added to pdf file ./fullimpacts.pdf
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: fullimpacts
Info in <TCanvas::Print>: Current canvas added to pdf file ./fullimpacts.pdf
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: fullimpacts
Info in <TCanvas::Print>: Current canvas added to pdf file ./fullimpacts.pdf
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: fullimpacts
Info in <TCanvas::Print>: Current canvas added to pdf file ./fullimpacts.pdf
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: fullimpacts
Info in <TCanvas::Print>: Current canvas added to pdf file ./fullimpacts.pdf
TCanvas::Constructor:0: RuntimeWarning: Deleting canvas with same name: fullimpacts
Info in <TCanvas::Print>: Current canvas 

In [17]:
from IPython.display import IFrame
IFrame("./fullimpacts.pdf", width=800, height=800)

Now we will compute the goodness of fit on observed using saturated algorithm

In [45]:
%%bash
combineTool.py -M GoodnessOfFit --algorithm saturated -m 125.09 -d comb_2017_htt_slim.root -n GOF_Saturated_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH 

 <<< Combine >>> 
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
Will use saturated models to compute goodness of fit for a binned likelihood
>>> method used is GoodnessOfFit
>>> random number generator seed is 123456
Set Range of Parameter r To : (0.3,1.7)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters

 --- GoodnessOfFit --- 
Best fit test statistic: 452.841
Done in 0.36 min (cpu), 0.36 min (re

And generate saturated distribution from 1000 toys using crab. Make sure you have initialized your grid proxy and that you have sourced the crab environment

In [46]:
%%bash
combineTool.py -M GoodnessOfFit --algorithm saturated -m 125.09 -d comb_2017_htt_slim.root -n GOF_Saturated_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --expectSignal 1.216 -s 1:1000:1 -t 1 --toysFrequentist --merge 10 --job-mode crab3 --task-name gof_saturated --custom-crab custom_crab.py

Traceback (most recent call last):
  File "/home/cms.zdemirag/CombineTutorial/CMSSW_8_1_0/bin/slc7_amd64_gcc530/combineTool.py", line 83, in <module>
    method.run_method()
  File "/home/cms.zdemirag/CombineTutorial/CMSSW_8_1_0/python/CombineHarvester/CombineTools/combine/EnhancedCombine.py", line 251, in run_method
    self.flush_queue()
  File "/home/cms.zdemirag/CombineTutorial/CMSSW_8_1_0/python/CombineHarvester/CombineTools/combine/CombineToolBase.py", line 279, in flush_queue
    from CRABAPI.RawCommand import crabCommand
  File "/cvmfs/cms.cern.ch/slc7_amd64_gcc530/lcg/root/6.06.08-giojec3/lib/ROOT.py", line 303, in _importhook
    return _orig_ihook( name, *args, **kwds )
ImportError: No module named CRABAPI.RawCommand


To get the crab results of the saturated GOF toys and add the output together, execute:

In [47]:
%%bash
#bash -c 'for f in /eos/uscms/store/user/${username}/Combine_Tutorial_2018/Combine/gof_saturated/*/*/combine_output_*.tar; do tar xf $f; done'
#hadd -f gof_saturated_toys.root higgsCombine*GOF_Saturated_htt_slim*.root

hadd Target file: gof_saturated_toys.root
hadd compression setting for all ouput: 1
hadd Source file 1: higgsCombineGOF_Saturated_htt_slim.GoodnessOfFit.mH125.09.root
hadd Target path: gof_saturated_toys.root:/
hadd Target path: gof_saturated_toys.root:/toys


tar: /eos/uscms/store/user//Combine_Tutorial_2018/Combine/gof_saturated/*/*/combine_output_*.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Now lets plot the goodness of fit, but first examine the plotting script.

In [48]:
%%bash
cat plotGOF.py

import ROOT

f = ROOT.TFile('gof_saturated_toys.root')
t = f.Get('limit')
c = ROOT.TCanvas()
vals = []
for e in range(t.GetEntries()):
    t.GetEntry(e)
    vals.append(t.limit)

t.Draw('limit>>hist(160,0,800)')
hist = ROOT.gDirectory.Get('hist')
hist.SetTitle('Test-statistic distribution')
hist.Draw("HIST")

fobs = ROOT.TFile('higgsCombineGOF_Saturated_htt_slim.GoodnessOfFit.mH125.09.root')
tobs = fobs.Get('limit')
tobs.GetEntry(0)
obs = tobs.limit

pval = sum(1.0 for i in vals if i >= obs) / float(len(vals))

arr = ROOT.TArrow(obs, 0.001, obs, hist.GetMaximum()/4, 0.02, "<|")
arr.SetLineColor(ROOT.kBlue)
arr.SetFillColor(ROOT.kBlue)
arr.SetFillStyle(1001)
arr.SetLineWidth(6)
arr.SetLineStyle(1)
arr.SetAngle(60)
arr.Draw("<|same")

c.SaveAs("saturated_gof.pdf")

print 'p-value for obs=%f is %f' % (obs, pval)


In [25]:
%%bash
python plotGOF.py -l -q -b

p-value for obs=452.841436 is 0.836000


Info in <TCanvas::Print>: pdf file saturated_gof.pdf has been created


In [26]:
from IPython.display import IFrame
IFrame("./saturated_gof.pdf", width=800, height=800)

To get postfit distributions we can run the  Fit Diagnostics

In [ ]:
%%bash
combine -M FitDiagnostics -m 125.09 -d comb_2017_htt_slim.root -n FitDiagnostics_htt_slim --setParameterRanges r=0.3,1.7 --cminDefaultMinimizerStrategy 1 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH

In [ ]:
%%bash
PostFitShapesFromWorkspace -w comb_2017_htt_slim.root -o comb_2017_htt_slim_shapes.root --samples 300 -f fitDiagnosticsFitDiagnostics_htt_slim.root:fit_s --postfit --sampling --skip-proc-errs

You can plot the postfit distributions using:

In [28]:
%%bash
cat plotPostFitShapes.py

import ROOT as r

r.gStyle.SetOptStat(0)
r.gStyle.SetOptTitle(0)

f = r.TFile("comb_2017_htt_slim_shapes.root","READ")

ch = "htt_mt_2_13TeV"

c1 = r.TCanvas(ch,ch,800,800)
c1.SetLogy()
c1.SetBottomMargin(0.4)
c1.cd()

total_bkg_pre = f.Get(ch+"_prefit/TotalBkg")
total_bkg_pre.SetLineColor(4)
total_bkg_pre.SetFillColor(4)
total_bkg_pre.SetMarkerColor(4)
total_bkg_pre.GetXaxis().SetTitle("Bin Number")
total_bkg_pre.GetYaxis().SetTitle("Number of Events")
total_bkg_pre.Draw("ae2")

total_bkg_post = f.Get(ch+"_postfit/TotalBkg")
total_bkg_post.SetLineColor(2)
total_bkg_post.SetFillColor(2)
total_bkg_post.Draw("e2same")


c1.SaveAs(ch+".pdf")

#others = []
#QCD



In [31]:
%%bash
python plotPostFitShapes.py -l -q -b

Info in <TCanvas::Print>: pdf file htt_mt_2_13TeV.pdf has been created


In [32]:
from IPython.display import IFrame
IFrame("./htt_mt_2_13TeV.pdf", width=800, height=800)

If you really crazy you can run full toy-based CLs limits on this complex model.
First check the results of Asymptotic Limits to get an idea of the limit. Event this takes about 10 minutes:

In [36]:
%%bash
combine -M Asymptotic -m 125.09 comb_2017_htt_slim.root --cminDefaultMinimizerStrategy 0 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --setParameterRanges r=0.0,2.2

 <<< Combine >>> 
 WARNING --  From combine v7, method Asymptotic has been renamed to AsymptoticLimits
>>> including systematics
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 0.1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 1, tolerance 1
Configured fallback algorithm , type Minuit2, algo Migrad, strategy 0, tolerance 1
>>> method used is Asymptotic
>>> random number generator seed is 123456
Set Range of Parameter r To : (0,2.2)
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)
ise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Enabling analytic minimisation of bin-wise statistical uncertainty parameters
>> Disabling analytic minimisation of bin-wise stati

You can run the toy Based CLs Limits on the grid

In [ ]:
%%bash
combineTool.py -M HyrbridNew --LHCMode LHC-limits -m 125.09 -d comb_2017_htt_slim.root -n FullCLs_htt_slim --cminDefaultMinimizerStrategy 1 --cminFallbackAlgo Minuit2,Migrad,0:0.1 --cminFallbackAlgo Minuit2,Migrad,1:1.0 --cminFallbackAlgo Minuit2,Migrad,0:1.0 --X-rtd MINIMIZER_MaxCalls=999999999 --X-rtd MINIMIZER_analytic --X-rtd FAST_VERTICAL_MORPH --clsAcc 0 -s 1:20:1 -T 100 --saveHybridResult --singlePoint 0.1:2.1:0.2 --job-mode crab3 --task-name full_cls --custom-crab custom_crab.py 

To collect the Toy Based CLs Limits from the grid and add the output together, you can execute:

In [ ]:
%%bash
#bash -c 'for f in /eos/uscms/store/user/${username}/Combine_Tutorial_2018/Combine/full_cls/*/*/combine_output*.tar; do tar -xvf $f; done'  
#hadd grid.root higgsCombineFullCLs_htt_slim.POINT.*.root 

You can finally compute the limits

In [33]:
%%bash
combine -M HybridNew -m 125.09 --LHCmode LHC-limits comb_2017_htt_slim.root -T 2000 --readHybridResults --grid grid.root --plot cls_fromgrid_exp025.pdf --expectedFromGrid 0.025
combine -M HybridNew -m 125.09 --LHCmode LHC-limits comb_2017_htt_slim.root -T 2000 --readHybridResults --grid grid.root --plot cls_fromgrid_exp16.pdf --expectedFromGrid 0.16
combine -M HybridNew -m 125.09 --LHCmode LHC-limits comb_2017_htt_slim.root -T 2000 --readHybridResults --grid grid.root --plot cls_fromgrid_exp50.pdf --expectedFromGrid 0.50
combine -M HybridNew -m 125.09 --LHCmode LHC-limits comb_2017_htt_slim.root -T 2000 --readHybridResults --grid grid.root --plot cls_fromgrid_exp84.pdf --expectedFromGrid 0.84
combine -M HybridNew -m 125.09 --LHCmode LHC-limits comb_2017_htt_slim.root -T 2000 --readHybridResults --grid grid.root --plot cls_fromgrid_exp975.pdf --expectedFromGrid 0.975
combine -M HybridNew -m 125.09 --LHCmode LHC-limits comb_2017_htt_slim.root -T 2000 --readHybridResults --grid grid.root --plot cls_fromgrid.pdf

 <<< Combine >>> 
>>> including systematics
>>> using the Profile Likelihood test statistics modified for upper limits (Q_LHC)
>>> method used is HybridNew
>>> random number generator seed is 123456
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)

 -- Hybrid New -- 
Limit: r < 0.201541 +/- 0.0315391 @ 95% CL
Done in 0.01 min (cpu), 0.01 min (real)
 <<< Combine >>> 
>>> including systematics
>>> using the Profile Likelihood test statistics modified for upper limits (Q_LHC)
>>> method used is HybridNew
>>> random number generator seed is 123456
>>> 0 out of 24 channels masked

Computing results starting from observation (a-posteriori)

 -- Hybrid New -- 
Limit: r < 0.292465 +/- 0.0171556 @ 95% CL
Done in 0.01 min (cpu), 0.01 min (real)
 <<< Combine >>> 
>>> including systematics
>>> using the Profile Likelihood test statistics modified for upper limits (Q_LHC)
>>> method used is HybridNew
>>> random number generator seed is 123456
>>> 0 out of 

Info in <TCanvas::Print>: pdf file cls_fromgrid_exp025.pdf has been created
Info in <TCanvas::Print>: pdf file cls_fromgrid_exp16.pdf has been created
Info in <TCanvas::Print>: pdf file cls_fromgrid_exp50.pdf has been created
Info in <TCanvas::Print>: pdf file cls_fromgrid_exp84.pdf has been created
Info in <TCanvas::Print>: pdf file cls_fromgrid_exp975.pdf has been created
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Reached call limit
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <Minuit2>: Minuit2Minimizer::Minimize : Minimization did NOT converge, Edm is above max
Info in <TCanvas::Print>: pdf file cls_fromgr

In [34]:
from IPython.display import IFrame
IFrame("./cls_fromgrid.pdf", width=800, height=800)

You can see that our observed limit is pretty accurate and is pretty close (within 2%) of the Aysmptotic limit. But we had to work a lot harder!